In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as sts
import statsmodels.api as sm
import warnings


from statsmodels.tsa.stattools import acf
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from scipy.stats.mstats import winsorize
from scipy.stats import jarque_bera
from scipy.stats import normaltest
from scipy.stats import bartlett
from scipy.stats import levene
from sqlalchemy import create_engine
from sklearn import linear_model

warnings.filterwarnings('ignore')

In [2]:
knt=pd.read_csv(".\\konut_train.csv")

kategorik=[i for i in knt.columns if knt[i].dtype=="object"]
surekli=[i for i in knt.columns if (knt[i].dtype=="int64" or knt[i].dtype=="float64")]

konut=knt
for i in kategorik:
    konut[i]=knt[i].fillna("Null")
for i in surekli:
    konut[i]=knt[i].fillna(0)

In [18]:
Y=konut["SalePrice"]
X=konut[["YearBuilt", "TotalBsmtSF", "TotRmsAbvGrd", "LotFrontage"]]

In [19]:
X=sm.add_constant(X)
sonuc=sm.OLS(Y, X).fit()
sonuc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.616
Model:                            OLS   Adj. R-squared:                  0.615
Method:                 Least Squares   F-statistic:                     582.6
Date:                Mon, 15 Apr 2019   Prob (F-statistic):          3.59e-300
Time:                        09:04:24   Log-Likelihood:                -17846.
No. Observations:                1460   AIC:                         3.570e+04
Df Residuals:                    1455   BIC:                         3.573e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.795e+06   9.07e+04    -19.779      0.000   -1.97e+06   -1.62e+06
YearBuilt      902.1209     46.553     19.378      0.000     810.803     993.439
TotalBsmtSF     65.4727      3.388     19.322      0.000      58.826      72.119
TotRmsAbvGrd  1.918e+04    840.426     22.819      0.000    1.75e+04    2.08e+04
LotFrontage     54.9156     38.957      1.410      0.159     -21.503     131.334
==============================================================================
Omnibus:                      412.541   Durbin-Watson:                   1.917
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14425.262
Skew:                           0.621   Prob(JB):                         0.00
Kurtosis:                      18.349   Cond. No.                     1.58e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

LotFrontage hariç diğer özelliklerimiz anlamlı, LotFrontage istatistiksel olarak anlamsızdır.

In [20]:
X=konut[["YearBuilt", "TotalBsmtSF", "TotRmsAbvGrd"]]

X=sm.add_constant(X)
sonuc=sm.OLS(Y, X).fit()
sonuc.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.615
Model:                            OLS   Adj. R-squared:                  0.614
Method:                 Least Squares   F-statistic:                     775.6
Date:                Mon, 15 Apr 2019   Prob (F-statistic):          3.20e-301
Time:                        09:07:02   Log-Likelihood:                -17847.
No. Observations:                1460   AIC:                         3.570e+04
Df Residuals:                    1456   BIC:                         3.572e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -1.786e+06   9.06e+04    -19.722      0.000   -1.96e+06   -1.61e+06
YearBuilt      898.1214     46.482     19.322      0.000     806.943     989.300
TotalBsmtSF     66.4089      3.324     19.980      0.000      59.889      72.929
TotRmsAbvGrd  1.937e+04    829.334     23.359      0.000    1.77e+04     2.1e+04
==============================================================================
Omnibus:                      419.569   Durbin-Watson:                   1.919
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            13730.612
Skew:                           0.669   Prob(JB):                         0.00
Kurtosis:                      17.964   Cond. No.                     1.58e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.58e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Katsayıların büyüklükleri değişti.

Bütün katsayılar anlamlıdır. P değerleri 0 (sıfır)'dır.

Evin fiyatınıen çok TotRmsAbvGrd (oda sayısı), en az TotalBsmtSF (feet kare) değişkeni etkilemektedir. Evin oda sayısı ve yapım yılı, fiyatını belirleme konusunda bariz baskın bir durumdadır.

Sonuçlar mantıklı, ancak metrekare (square kare) büyüklüğünün daha fazla etkiye sahip olmasını beklerdim.